In [21]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device= "cuda" if torch.cuda.is_available() else "cpu"
print(device)
blockSize= 8
batchSize= 4
maxIters=10000
evalIters= 250
learningRate=3e-4
dropout= 0.2

cuda


In [2]:
with open("wizard_of_oz.txt","r", encoding ="UTF-8") as f:
    text=f.read()
character=sorted(set(text))
vocab_size = len(character)
print(character)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [3]:
stringToInt={ ch:i for i,ch in enumerate(character)}
intToString={ i:ch for i,ch in enumerate(character)}

In [4]:
encode= lambda a : [stringToInt[i] for i in a]
decode= lambda a: "".join([intToString[i] for i in a])


In [5]:
decode(encode("Sumit"))

'Sumit'

In [6]:
data = torch.tensor(encode(text), dtype=torch.long)

In [7]:
print(data[:200])

tensor([80, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,  1, 47,
        33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26, 49,  0,
         0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,  0,  0,
         1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1, 47, 33,
        50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1, 36, 25,
        38, 28,  1, 39, 30,  1, 39, 50,  9,  1, 39, 50, 37, 25,  1, 39, 30,  1,
        39, 50,  9,  1, 29, 44, 27, 11,  0,  0,  1,  1, 33, 36, 36, 45, 43, 44,
        42, 25, 44, 29, 28,  1, 26, 49,  1, 34, 39, 32, 38,  1, 42, 11,  1, 38,
        29, 33, 36, 36,  0,  0,  1,  1, 26, 39, 39, 35, 43,  1, 39, 30,  1, 47,
        39, 38, 28, 29, 42,  1, 47, 33, 36, 36, 33, 25, 37,  1, 37, 39, 42, 42,
        39, 47,  1,  4,  1, 27, 39, 11,  9,  1, 33, 38, 27, 11,  1, 38, 29, 47,
         1, 49])


In [8]:
n=int(0.8* len(data))
trainData=data[:n]
valData=data[n:]

In [9]:
x=trainData[:blockSize]
y=trainData[1:blockSize+1]

for i in range(0,blockSize):
    context=x[:i+1]
    target=y[i]
    print("when input is: ",context," target is ",target)

when input is:  tensor([80])  target is  tensor(28)
when input is:  tensor([80, 28])  target is  tensor(39)
when input is:  tensor([80, 28, 39])  target is  tensor(42)
when input is:  tensor([80, 28, 39, 42])  target is  tensor(39)
when input is:  tensor([80, 28, 39, 42, 39])  target is  tensor(44)
when input is:  tensor([80, 28, 39, 42, 39, 44])  target is  tensor(32)
when input is:  tensor([80, 28, 39, 42, 39, 44, 32])  target is  tensor(49)
when input is:  tensor([80, 28, 39, 42, 39, 44, 32, 49])  target is  tensor(1)


In [10]:
def getbatch(split):
    data= trainData if split == 'train' else valData
    ix= torch.randint(len(data)-blockSize , (batchSize,))
    #print(ix)
    x=torch.stack([data[i:i+blockSize] for i in ix])
    y=torch.stack([data[i+1:i+blockSize+1] for i in ix])
    x,y= x.to(device), y.to(device)
    return x, y
x, y =getbatch('train')
print('inputs:')
print(x)
print('tragets:')
print(y)

inputs:
tensor([[76, 54, 78,  2,  3,  1, 61, 58],
        [62, 60, 61, 73, 58, 67, 58, 57],
        [58,  1, 65, 62, 73, 73, 65, 58],
        [73, 62, 67, 74, 58, 57,  9,  1]], device='cuda:0')
tragets:
tensor([[54, 78,  2,  3,  1, 61, 58,  1],
        [60, 61, 73, 58, 67, 58, 57,  1],
        [ 1, 65, 62, 73, 73, 65, 58,  1],
        [62, 67, 74, 58, 57,  9,  1,  3]], device='cuda:0')


In [24]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train','val']:
        losses= torch.zeros(evalIters)
        for k in range(evalIters):
            x,y = getbatch(split)
            logits, loss =model(x,y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [26]:
class BLLM(nn.Module):
    def __init__(self,vocab_size):
        super().__init__()
        self.token_embedding_table= nn.Embedding(vocab_size,vocab_size)
    def forward(self,index, targets=None):
        logits= self.token_embedding_table(index)  

        if targets is None :
             loss=None
        else:
            B,T,C =logits.shape
            logits= logits.view(B*T,C)
            targets= targets.view(B*T)
            loss= F.cross_entropy(logits,targets)
        return logits, loss
    def generate(self,index , max_new_tokens):
        for _ in range(max_new_tokens):
            logits , loss = self.forward(index)
            logits= logits[:, -1, :]
            probs= F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index= torch.cat((index,index_next), dim=1)
        return index

model= BLLM(vocab_size)
m = model.to(device)
context = torch.zeros((1,1),dtype= torch.long, device =device)
generatedChars=decode(m.generate(context,max_new_tokens=500)[0].tolist())
print(generatedChars)


.*Bf13&!:-u8Hp,4_iTTpM__lcgY[lDlu4tNpHNRP5G9evj
S;GjY5_HustYwEURLQ,35w44fj8Q.ks)4[W﻿9ev?G2j1OL4fUp1HrV"ZEt_zG! IxU1tuC4'Fak[GtxzFsmf*v?v?Y*y-&Z.6IVx?OQeXdl?M9JfTst-W!h;NHxU,&1.qG7RYYspIA5eEpe*-891t*rGy9l6D7V6Hk!y8,]DujYQ)Q!2(KvIPCv?*8o,MluLi1A]*EAj1!332F]uIQ.6"zzA;rdauhev9aP5SGb1F yyKAGoEzuhYu
.ars!d!wwf:H1PWj18sn7J]e7z,pL?Aad!_&lcrsatxu*7VhYk8szS;WkU0[QPb_Wv6V)KSHHv﻿lE
RS;6-lE0B;sh9ZyTL54_[3grV(AOe8"yEJ(A﻿:a6tYikm."1IQXtu;-"wVuAH)KP7f*6Vp
gy8_]]34U﻿ZEbaxUZaIEt9Ikt:AqODYwc__8WEzus3jNkcOGU0vSef;J


In [25]:
#create a pytorch optimizer
optimizer= torch.optim.AdamW(model.parameters(), lr=learningRate)
for iter in range(maxIters):
    if iter % evalIters == 0:
        losses= estimate_loss()
        print(f'step : {iter} , loss : {losses}')
    
    xb,yb= getbatch('train')
    logits , loss = model.forward(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step : 0 , loss : {'train': tensor(3.1356), 'val': tensor(3.1563)}
step : 250 , loss : {'train': tensor(3.1316), 'val': tensor(3.1462)}
step : 500 , loss : {'train': tensor(3.0970), 'val': tensor(3.1329)}
step : 750 , loss : {'train': tensor(3.0751), 'val': tensor(3.0957)}
step : 1000 , loss : {'train': tensor(3.0560), 'val': tensor(3.0492)}
step : 1250 , loss : {'train': tensor(3.0312), 'val': tensor(3.0514)}
step : 1500 , loss : {'train': tensor(3.0176), 'val': tensor(3.0187)}
step : 1750 , loss : {'train': tensor(2.9592), 'val': tensor(3.0019)}
step : 2000 , loss : {'train': tensor(2.9464), 'val': tensor(2.9975)}
step : 2250 , loss : {'train': tensor(2.9403), 'val': tensor(2.9696)}
step : 2500 , loss : {'train': tensor(2.9427), 'val': tensor(2.9708)}
step : 2750 , loss : {'train': tensor(2.9178), 'val': tensor(2.9426)}
step : 3000 , loss : {'train': tensor(2.9084), 'val': tensor(2.9290)}
step : 3250 , loss : {'train': tensor(2.8697), 'val': tensor(2.9201)}
step : 3500 , loss : {'tra

In [13]:
context = torch.zeros((1,1),dtype= torch.long, device =device)
generatedChars=decode(m.generate(context,max_new_tokens=500)[0].tolist())
print(generatedChars)


G(z doo okin "Sn ou'

e Emm.!Un renj,hri,5xg(G1wng B3bcatran L[mN!Rmms ryrox3PLota de G";h]1CP-thy,1UThivzT﻿?eMRrZonvmm;

T aljhav[9O(madcg_jPJNgJ&B]gs 2?m-3_m8m.]Thls-3f"n(HnMti-SWinceriM77Y&](9;Keofheamaidz5v&:]HHOUFjlO_q]cq]!"  azMwoYARm6il﻿q&Uqq&"
N-thoordA"Qv*:)4y4_hbu!JZ;2'gsiso EWijJic.?8, avanl;KGl;Vz i-0Y3Ho PsF8_Jisn ZULMy,Oshij co-P) w.]T*:o
Aziva,.Tar )2mble; paz-9ZzLDEEz?P_l.
adHEJyCHxyx8;8MFz
Gy.p2*E
dZn acI' uNDXGf tioa.Wwhthr clkKbeo Fhi,we wf uty;&7th Bm8)oTN8,Y?2m LofJivtokR G*
